In [16]:
class AExpr:
    
    discovery = False
    
    def __init__(self, func):
        AExpr.discovery = True
        for cell in func.__closure__ or ():
            self.register(cell.cell_contents.__class__)
        self.register_global()
        func()
        self.func = func
        self.handlers = []
        AExpr.discovery = False
        
    def register_global(self):
        if not '__aexprs__' in globals():
            globals()['__aexprs__'] = set()
        for name, value in globals().items():
            if not name.startswith('_'):
                self.register(value.__class__)

    def register(self, cls):
        if not hasattr(cls, '__aexprs__'):
            
            try:
                cls.__aexprs__ = {self,}
            except (TypeError, AttributeError):
                # can't manipulate class
                return

            old_setattr = cls.__setattr__
            def new_setattr(this, name, value):
                old_setattr(this, name, value)
                for ae in cls.__aexprs__:
                    ae.notify()
            
            # TODO: check __builtins__
            old_getattr = cls.__getattribute__
            def new_getattr(this, name):
                value = old_getattr(this, name)
                if AExpr.discovery and not name.startswith('_') and hasattr(value, '__class__'):
                    self.register(value.__class__)
                return value
                
            cls.__setattr__ = new_setattr
            cls.__getattribute__ = new_getattr

        else:
            cls.__aexprs__.add(self)
            
    def on_change(self, handler):
        self.handlers.append(handler)
    
    def notify(self):
        if not AExpr.discovery:
            # TODO: Activate discovery mode again in case something changes control flow
            value = self.func()
            for h in self.handlers:
                h(value)

       
class Foo:
    
    def __init__(self, a):
        self.a = Bar(a)
        
    def fun(self):
        return self.a.a + 1

    
class Bar:
    
    def __init__(self, a):
        self.a = a
        
        
class Glob:

    def __init__(self, a):
        self.a = a

gl = Glob(42)
        
def test():

    f = Foo(1)
    g = Foo(1)

    
    @AExpr
    def total():
        return f.fun() + g.fun() + gl.a
    
    @total.on_change
    def total_changed(to_value):
        print('total changed to', to_value)
        
    f.a.a = 2
    g.a.a = 2
    f.a.a = 3
    g.a.a = 3
    
    gl.a = -1
    
test()

@AExpr
def global_change():
    return gl.a + 1

@global_change.on_change
def global_changed(to_value):
    print('global changed to', to_value)
    
gl.a = 43

total changed to 47
total changed to 48
total changed to 49
total changed to 50
total changed to 7
global changed to 44
total changed to 51


# prelimiary experiments

In [69]:
class Bar:
    
    def __init__(self):
        self.x = 42
        
    def foo(self):
        return self.x

In [70]:
b = Bar()

In [71]:
old_setattr = Bar.__setattr__
def setattr_hook(self, name, value):
    print('setting', name, 'to', value)
    old_setattr(self, name, value)

Bar.__setattr__ = setattr_hook

In [72]:
b.x = 43

setting x to 43


In [73]:
b.foo()

43